# Çoklu Lineer Regresyon ve Gradient Descent Algoritması ile tahmin

In [1]:
import math , copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
nba = pd.read_csv('vised_zscaled_nba_stats_salaries_2017_2018.csv')

nba

,Player,Stats_from,Age,MPG,PPG,PER,TOPG,PFG,2017_2018 Salary
0,Quincy Acy,2016.0,0.30,0.355789,0.173081,0.564232,0.100335,0.480824,1.709538
1,Steven Adams,2016.0,0.15,0.641793,0.264442,0.584383,0.230214,0.767744,22.471910
2,Arron Afflalo,2016.0,0.55,0.869415,0.425862,0.468514,0.253220,0.550847,2.328652
3,Alexis Ajinca,2016.0,0.40,0.348748,0.198451,0.541562,0.200671,0.625539,4.961798
4,Cole Aldrich,2016.0,0.40,0.313858,0.181839,0.730479,0.233868,0.638065,7.300000
...,...,...,...,...,...,...,...,...,...
340,Joe Young,2016.0,0.20,0.203985,0.124940,0.443325,0.176471,0.201530,1.471382
341,Nick Young,2016.0,0.55,0.474377,0.241466,0.423174,0.121806,0.255022,5.192000
342,Thaddeus Young,2016.0,0.40,0.857735,0.502137,0.634761,0.408468,0.686673,14.796348
343,Cody Zeller,2016.0,0.20,0.617600,0.290711,0.599496,0.204234,0.769677,12.584270


In [3]:
nba.drop(columns=['Player', 'Stats_from'], axis=1, inplace=True)

nba

,Age,MPG,PPG,PER,TOPG,PFG,2017_2018 Salary
0,0.30,0.355789,0.173081,0.564232,0.100335,0.480824,1.709538
1,0.15,0.641793,0.264442,0.584383,0.230214,0.767744,22.471910
2,0.55,0.869415,0.425862,0.468514,0.253220,0.550847,2.328652
3,0.40,0.348748,0.198451,0.541562,0.200671,0.625539,4.961798
4,0.40,0.313858,0.181839,0.730479,0.233868,0.638065,7.300000
...,...,...,...,...,...,...,...
340,0.20,0.203985,0.124940,0.443325,0.176471,0.201530,1.471382
341,0.55,0.474377,0.241466,0.423174,0.121806,0.255022,5.192000
342,0.40,0.857735,0.502137,0.634761,0.408468,0.686673,14.796348
343,0.20,0.617600,0.290711,0.599496,0.204234,0.769677,12.584270


# Training datasını load etme

In [4]:
y_train = nba['2017_2018 Salary']

y_train.shape

(345,)

In [5]:
nba_X = nba.drop('2017_2018 Salary',axis = 1)

X_train = nba_X.to_numpy()

X_train.shape

(345, 6)

# Cost Function:

    J(w,b) = 1/2m( f_wb(x ^ i) - y ^ i) ^ 2  ;  i = 0..m - 1

    ve f_wb(x) = w * x[i] + b olmak üzere:

In [6]:
def compute_cost(X, y, w, b): 
    
    m = X.shape[0]
    cost = 0.0
    for i in range(m):                                
        f_wb_i = np.dot(X[i], w) + b           
        cost += (f_wb_i - y[i])**2     
    
    cost /= (2 * m)                      
    return cost

In [7]:
b_test = 0.2
w_test = np.array([0.1 , 0.2 , 0.22 , 0.5 , -0.025 , -0.01])
compute_cost(X_train,y_train,w_test,b_test)

62.805756893464675

# Gradient Descent Algoritması:

yakınsama olana kadar tekrar et : {

    w_j = w_j - a * (dJ(w,b) / dw_j)  ; j = 0..n - 1 (n = feature sayısı , a = learning rate alpha)

    b = b - a * (dJ(w,b) / db)                     (w_j ve b aynı anda update edilecek(simultaneously))

}

ve 

    (dJ(w,b) / dw_j) = 1/m( f_wb(x ^ i) - y ^ i)x_j ^ i  ; i = 0..m - 1

    (dJ(w,b) / db) =  1/m( f_wb(x ^ i) - y ^ i)

In [8]:
# Gradient descent formüllerindeki türevli kısımları hesaplar

def compute_gradient_derivatives(X, y, w, b): 
   
    m,n = X.shape           
    dj_dw = np.zeros((n,))  # J nin w ye göre kısmi türevi
    dj_db = 0.              # J nin b ye göre kısmi türevi

    for i in range(m):                             
        
        error = (np.dot(X[i], w) + b) - y[i]  
        
        for j in range(n):                         
            
            dj_dw[j] = dj_dw[j] + error * X[i, j]    
            
        dj_db = dj_db + error  
        
    dj_dw = dj_dw / m
    
    dj_db = dj_db / m  
                                    
    return dj_db, dj_dw

In [9]:
compute_gradient_derivatives(X_train,y_train,w_test,b_test)

(-8.023162285520222,
 array([-3.00178639, -5.81837496, -3.66659903, -5.00644721, -3.06875504,
        -4.81854152]))

In [10]:
# Gradient descent ile J yi minimize edecek w,b değerlerini bulur, iterasyon sayısını ve w,b geçmişini gösterir

def gradient_descent(X, y, w_in, b_in, cost_function, compute_gradient_derivatives, alpha, num_iters): 
    
    J_history = []
    w = copy.deepcopy(w_in)  #fonksiyondaki global w değişkenin değiştirmemek için
    b = b_in
    
    for i in range(num_iters):

        # Türevleri hesapla ve ata
        dj_db,dj_dw = compute_gradient_derivatives(X, y, w, b)

        # Gradient Descent Algoritması, eş zamanlı update ediliyor
        w = w - alpha * dj_dw             
        b = b - alpha * dj_db
      
        # Her iterasyon sonrası cost değerlerini kaydet
        if i<100001:      # sınırlama 
            J_history.append(cost_function(X, y, w, b))

        # yapılacak iterasyon sayısının 10'da 1'ine gelince J_history'nin son elemanını yazdır
        if i% math.ceil(num_iters / 10) == 0:  
            print(f"Iteration {i}: Cost {J_history[-1]}")
        elif num_iters == len(J_history):
            print(f"Iteration {num_iters}: Cost {J_history[-1]}")
        
    return w, b, J_history

In [11]:
w_init = np.zeros(6)
b_init = 0
iterations = 1000
alpha = 0.01

w_final , b_final , J_hist = gradient_descent(
                X_train, y_train, w_init, b_init , compute_cost, compute_gradient_derivatives, alpha, iterations)

print(f"b,w found by gradient descent: {b_final},{w_final} ")

for i in range(10):
    print(f"prediction: {np.dot(X_train[i], w_final) + b_final:0.1f}, target value: {y_train[i]:0.1f}")

Iteration 0: Cost 67.25141478915107
Iteration 100: Cost 23.86883089952494
Iteration 200: Cost 22.54555596284038
Iteration 300: Cost 21.734710867181846
Iteration 400: Cost 21.0326188342363
Iteration 500: Cost 20.421426671017876
Iteration 600: Cost 19.888805043300195
Iteration 700: Cost 19.424138640992116
Iteration 800: Cost 19.018262734743832
Iteration 900: Cost 18.663264078213857
Iteration 1000: Cost 18.35522100877221
b,w found by gradient descent: -0.49194152319425777,[0.3251994  5.99537908 5.79845497 2.99565515 3.68121144 2.41273204] 
prediction: 6.0, target value: 1.7
prediction: 9.4, target value: 22.5
prediction: 11.0, target value: 2.3
prediction: 6.8, target value: 5.0
prediction: 7.2, target value: 7.3
prediction: 12.6, target value: 21.5
prediction: 9.6, target value: 2.1
prediction: 10.1, target value: 7.3
prediction: 4.3, target value: 1.6
prediction: 5.7, target value: 2.2


# Sonuç

    Bazı feature'lar lineer regresyona pek uygun olmamasına rağmen devam etmek ve özelliklerin katsayılarını yorumlamak istedim. Farklı learning rate ler, farklı iterasyon sayıları denememe rağmen çok isabetli denebilecek tahminler elde edemedim beklendiği üzere.

    Pozitif katsayılar büyükten küçüğe : PPG > MPG > PER > Age --> Buradan maaşlara en çok etkisi olan özelliklerin sırasıyla 'Maç başına atılan sayı' , 'Maç başına alınan süre' , 'Oyuncu verimlilik puanı' ve 'Yaş' olduğu söylenebilir.

    Negatif özelliklerde ise ; 'PFG' katsayısı 'TOPG' e kıyasla daha fazla olduğu için, maç başına yapılan faullerin maaşlara top kayıplarına göre daha çok etki ettiği söylenebilir.